## Práctica 3 - Deep Learning: Red Neuronal Convolucional (CNN)

***
**Jaime Alonso Fernández**  
**Universidad Complutense de Madrid**  
**Grado en Ingeniería del Software (Plan 2019)**  
**Asignatura de Aprendizaje Automático y Big Data**
***

